# Attention Architecture

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import music21

In [3]:
# from fastai.text import *
from enum import Enum
import torch
from fastai.text.models.awd_lstm import *
from fastai.text.models.transformer import *

In [4]:
import numpy as np
import torch.nn as nn

In [5]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [6]:
import sys
sys.path.insert(0, '../../')
from src.fastai_data import *
from src.encode_data import *
from src.serve import *

In [7]:
from src.lmnp_transformer import *

In [8]:
path = Path('../../data/midi/v14/piano_duet/')

## Single Stream Encoding

In [9]:
config = v14s_config(vocab); config

{'ctx_len': 150,
 'n_layers': 16,
 'n_heads': 8,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 512,
 'mask': True,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 16,
 'bptt': 256,
 'vocab_size': 274}

## Fastai Learner

In [11]:
dl_tfms = [mask_tfm, next_sentence_tfm]

In [12]:
data = load_music_data(path, cache_name='tmp/sample', vocab=vocab, y_offset=0, dl_tfms=dl_tfms, **config)

DLTFMS: [<function mask_tfm at 0x7f22600c6620>, <function next_sentence_tfm at 0x7f22600c6598>]


In [13]:
xb,yb = data.one_batch(cpu=False)

In [14]:
xb

tensor([[141,   8, 141,  ...,  96, 143,  86],
        [141,   8, 141,  ...,  94, 143,  84],
        [141,   8, 141,  ...,  99, 143,  89],
        ...,
        [141,   8, 141,  ...,  96, 143,  86],
        [141,   8, 141,  ...,  95, 143,  85],
        [  4,   8, 141,  ...,  96, 143,  86]], device='cuda:0')

In [15]:
yb

[tensor([[  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [141,   1,   1,  ...,   1,   1,   1]], device='cuda:0'),
 tensor([[0, 0, 0,  ..., 2, 2, 2],
         [0, 0, 0,  ..., 2, 2, 2],
         [0, 0, 0,  ..., 2, 2, 2],
         ...,
         [0, 0, 0,  ..., 2, 2, 2],
         [0, 0, 0,  ..., 2, 2, 2],
         [0, 0, 0,  ..., 2, 2, 2]], device='cuda:0')]

## LMNP

In [16]:
# m_len = 0
# x_len = 16 # bptt
# seq_len = m_len+x_len
# torch.triu(torch.ones(x_len, seq_len), diagonal=m_len).byte()[None,None].cpu().numpy()
# torch.triu(torch.ones(x_len, seq_len), diagonal=m_len+1).byte()[None,None].cpu().numpy()

In [17]:
class MusicAttention(MultiHeadRelativeAttention):
    def __init__(self, n_heads:int, d_model:int, d_head:int, resid_p:float=0., attn_p:float=0., bias:bool=True,
                 scale:bool=True, residual:bool=True):
        super().__init__(n_heads, d_model, d_head, resid_p=resid_p, attn_p=attn_p, bias=bias, scale=scale)
        self.residual = residual
        
    def forward(self, x:Tensor, mask:Tensor=None, **kwargs):
        if self.residual: return self.ln(x + self.drop_res(self.out(self._apply_attention(x, mask=mask, **kwargs))))
        return self.ln(self.drop_res(self.out(self._apply_attention(x, mask=mask, **kwargs))))
    
class MusicTransformerXL(nn.Module):
    "TransformerXL model: https://arxiv.org/abs/1901.02860."
    def __init__(self, vocab_sz:int, ctx_len:int, n_layers:int, n_heads:int, d_model:int, d_head:int, d_inner:int, 
                 resid_p:float=0., attn_p:float=0., ff_p:float=0., embed_p:float=0., bias:bool=False, scale:bool=True,
                 act:Activation=Activation.ReLU, double_drop:bool=True, attn_cls:Callable=MusicAttention,
                 learned_pos_enc:bool=False, mask:bool=True, mem_len:int=0, **kwargs):
        super().__init__()
        self.encoder = nn.Embedding(vocab_sz, d_model)
        self.pos_enc = nn.Embedding(ctx_len, d_model) if learned_pos_enc else PositionalEncoding(d_model)
        self.drop_emb = nn.Dropout(embed_p)
        self.u = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.v = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.mem_len,self.n_layers,self.d_model,self.mask = mem_len,n_layers,d_model,mask
        self.init = False
        self.layers = nn.ModuleList([DecoderLayer(n_heads, d_model, d_head, d_inner, resid_p=resid_p, attn_p=attn_p,
                      ff_p=ff_p, bias=bias, scale=scale, act=act, double_drop=double_drop, 
                      attn_cls=attn_cls) for k in range(n_layers)])
#         self.layers[0].mhra.residual = False
    
    def reset(self):
        "Reset the internal memory."
        self.hidden = [next(self.parameters()).data.new(0) for i in range(self.n_layers+1)]

    def _update_mems(self, hids):
        if not getattr(self, 'hidden', False): return None
        assert len(hids) == len(self.hidden), 'len(hids) != len(self.hidden)'
        with torch.no_grad():
            for i in range(len(hids)):
                cat = torch.cat([self.hidden[i], hids[i]], dim=1)
                self.hidden[i] = cat[:,-self.mem_len:].detach()
    
    def select_hidden(self, idxs): self.hidden = [h[idxs] for h in self.hidden]
    
    def forward(self, x):
        #The hidden state has to be initiliazed in the forward pass for nn.DataParallel
        if self.mem_len > 0 and not self.init: 
            self.reset()
            self.init = True
        bs,x_len = x.size()
        inp = self.drop_emb(self.encoder(x)) #.mul_(self.d_model ** 0.5)
        m_len = self.hidden[0].size(1) if hasattr(self, 'hidden') and len(self.hidden[0].size()) > 1 else 0
        seq_len = m_len + x_len
#         mask = torch.triu(x.new_ones(x_len, seq_len), diagonal=m_len+1).byte()[None,None] if self.mask else None
        mask = torch.triu(x.new_ones(x_len, seq_len), diagonal=m_len).byte()[None,None] if self.mask else None
        if m_len == 0 and self.mask: mask[0,0] = 0
        #[None,:,:None] for einsum implementation of attention
        hids = []
        pos = torch.arange(seq_len-1, -1, -1, device=inp.device, dtype=inp.dtype)
        pos_enc = self.pos_enc(pos)
        hids.append(inp)
        for i, layer in enumerate(self.layers):
            mem = self.hidden[i] if self.mem_len > 0 else None
            inp = layer(inp, r=pos_enc, u=self.u, v=self.v, mask=mask, mem=mem)
            hids.append(inp)
        core_out = inp[:,-x_len:]
        if self.mem_len > 0 : self._update_mems(hids)
        return (self.hidden if self.mem_len > 0 else [core_out]),[core_out]


In [18]:
# learn.model[0].layers[0].mhra.residual

In [19]:
from fastai.text.learner import _model_meta

In [20]:
config['mem_len'] = 0
config['mask'] = False

In [21]:
config

{'ctx_len': 150,
 'n_layers': 16,
 'n_heads': 8,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 0,
 'mask': False,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 16,
 'bptt': 256,
 'vocab_size': 274}

In [22]:
_model_meta[MusicTransformerXL] = _model_meta[TransformerXL]
_model_meta[MusicTransformerXL]['config_lm'] = config

In [24]:

class LinearDecoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, output_p:float, tie_encoder:nn.Module=None, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1])
        decoded = self.decoder(output)
        return decoded, raw_outputs, outputs


In [25]:
class BertHead(nn.Module):
    def __init__(self, encoder, decoder, ns_decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.ns_decoder = ns_decoder
        
    def forward(self, x):
        x_e = self.encoder(x)
        return self.decoder(x_e), self.ns_decoder(x_e)
    
    def __getitem__(self, idx):
        return [self.encoder, self.decoder, self.ns_decoder][idx]
        
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in [self.encoder, self.decoder, self.ns_decoder]:
            if hasattr(c, 'reset'): c.reset()

In [26]:

class NSDecoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        if bias: self.decoder.bias.data.zero_()

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        raw_outputs, outputs = input
        decoded = self.decoder(outputs[-1])
        return decoded, raw_outputs, outputs

In [27]:
def get_music_model(arch:Callable, vocab_sz:int, config:dict=None, drop_mult:float=1.):
    "Create a language model from `arch` and its `config`, maybe `pretrained`."
    meta = _model_meta[arch]
    config = ifnone(config, meta['config_lm'].copy())
    for k in config.keys(): 
        if k.endswith('_p'): config[k] *= drop_mult
#     tie_weights,output_p,out_bias = map(config.pop, ['tie_weights', 'output_p', 'out_bias'])
    tie_weights,output_p,out_bias = map(config.get, ['tie_weights', 'output_p', 'out_bias'])
    init = config.pop('init') if 'init' in config else None
    encoder = arch(vocab_sz, **config)
    enc = encoder.encoder if tie_weights else None
    decoder = LinearDecoder(vocab_sz, config[meta['hid_name']], output_p, tie_encoder=enc, bias=out_bias)
    ns_decoder = NSDecoder(4, config[meta['hid_name']])
    model = BertHead(encoder, decoder, ns_decoder)
    return model if init is None else model.apply(init)


def music_model_learner(arch:Callable, data:DataBunch, config:dict=None, drop_mult:float=1., pretrained:bool=False,
                        pretrained_fnames:OptStrTuple=None, **learn_kwargs) -> 'LanguageLearner':
    "Create a `Learner` with a language model from `data` and `arch`."
    model = get_music_model(arch, config['vocab_size'], config=config, drop_mult=drop_mult)
    
    meta = _model_meta[arch]
    learn = MusicLearner(data, model, split_func=meta['split_lm'], 
                         bos_idx=config['bos_idx'], sep_idx=config['sep_idx'],
                        **learn_kwargs)
    
    if pretrained:
        if 'url' not in meta: 
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta['url'], data=False)
        fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn.load_pretrained(*fnames)
        learn.freeze()
    if pretrained_fnames is not None:
        fnames = [learn.path/learn.model_dir/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]
        learn.load_pretrained(*fnames)
        learn.freeze()
    return learn

## Load

In [28]:
learn = music_model_learner(MusicTransformerXL, data, config, drop_mult=0)

Sep_idx: 8


In [29]:
xb,yb = data.one_batch(cpu=False)

In [30]:
# xb.shape, yb.shape

In [31]:
class BertLoss():
    def __init__(self, mask_loss, sent_loss):
        self.mask_loss = mask_loss
        self.sent_loss = sent_loss
        
    def __call__(self, input:Tensor, target:Tensor, target_sen:Tensor, **kwargs)->Rank0Tensor:
        m = self.mask_loss.__call__(input[0], target, **kwargs)
        s = self.sent_loss.__call__(input[1], target_sen, **kwargs)
        return m + s

In [32]:
from fastai.callbacks.rnn import RNNTrainer
class BertRNNTrainer(RNNTrainer):
    "`Callback` that regroups lr adjustment to seq_len, AR and TAR."
    def __init__(self, learn:Learner, alpha:float=0., beta:float=0.):
        super().__init__(learn, alpha, beta)
        
#     def on_epoch_begin(self, **kwargs):
#         "Reset the hidden state of the model."
#         self.learn.model.reset()

    def on_loss_begin(self, last_output:Tuple[Tensor,Tensor,Tensor], **kwargs):
        "Save the extra outputs for later and only returns the true output."
        return super().on_loss_begin(last_output[0])
#         self.raw_out,self.out = last_output[1],last_output[2]
#         return {'last_output': last_output[0]}

#     def on_backward_begin(self, last_loss:Rank0Tensor, last_input:Tensor, **kwargs):
#         "Apply AR and TAR to `last_loss`."
#         #AR and TAR
#         if self.alpha != 0.:  last_loss += self.alpha * self.out[-1].float().pow(2).mean()
#         if self.beta != 0.:
#             h = self.raw_out[-1]
#             if len(h)>1: last_loss += self.beta * (h[:,1:] - h[:,:-1]).float().pow(2).mean()
#         return {'last_loss': last_loss}

In [33]:
class BertTrainer(LearnerCallback):
    "`Callback` that regroups lr adjustment to seq_len, AR and TAR."
    def __init__(self, learn:Learner):
        super().__init__(learn)
        
    def on_loss_begin(self, last_output:Tuple[Tensor,Tensor,Tensor], **kwargs):
        "Save the extra outputs for later and only returns the true output."
        return {'last_output': (last_output[0][0], last_output[1][0]) }


In [34]:
type(learn.callbacks)

list

In [35]:
# learn.callbacks = [BertTrainer(learn, alpha=2, beta=1)]
learn.callbacks = [BertTrainer(learn)]

In [36]:
learn.loss_func = BertLoss(CrossEntropyFlat(ignore_index=vocab.pad_idx), CrossEntropyFlat())

In [37]:
learn.model(xb)

((tensor([[[ 0.0960,  0.7304,  0.5870,  ..., -0.3558, -0.0351,  0.1293],
           [ 0.0330, -0.2509,  0.1509,  ...,  0.1848,  0.1848, -0.1398],
           [-0.3860,  0.5528,  0.1872,  ...,  0.0809,  0.3503,  0.1579],
           ...,
           [ 0.1481, -0.5409, -0.2819,  ..., -0.4559,  0.2345, -0.4213],
           [ 0.1694, -0.2258, -0.7146,  ..., -0.0030, -0.1551, -0.2437],
           [-0.3028, -0.4687, -0.1243,  ..., -0.4610, -0.1998,  0.1170]],
  
          [[ 0.1549, -0.5216, -0.2756,  ..., -0.4585,  0.2363, -0.4137],
           [ 0.0422, -0.2308,  0.1545,  ...,  0.1814,  0.1888, -0.1338],
           [ 0.3883,  0.0983,  0.1274,  ..., -0.6362,  0.3298, -0.1741],
           ...,
           [ 0.1551, -0.5209, -0.2770,  ..., -0.4577,  0.2364, -0.4144],
           [-0.0859,  0.2476, -0.3591,  ...,  0.1663, -0.0769,  0.4624],
           [-0.2974, -0.4600, -0.1151,  ..., -0.4643, -0.1916,  0.1223]],
  
          [[-0.0865, -0.0054,  0.1733,  ...,  0.0117, -0.1034, -0.3831],
           

In [49]:
def mask_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    n = t1.shape[0]
    input = input[0].argmax(dim=-1).view(n,-1)
    t1 = t1.view(n,-1)
    mask = t1 != vocab.pad_idx
    return (input[mask]==t1[mask]).float().mean()

def ns_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    return accuracy(input[1], t2[0])

In [50]:
learn.metrics = [mask_acc, ns_acc]

In [51]:
learn.validate()

[2.2520301, tensor(0.5954), tensor(0.6895)]

In [41]:
# learn.lr_find()
# learn.recorder.plot()

In [52]:
learn.fit_one_cycle(3, 1e-4)

epoch,train_loss,valid_loss,mask_acc,ns_acc,time
0,2.417764,2.317451,0.598299,0.429834,01:14
1,2.165608,2.128812,0.608285,0.476660,01:14
2,2.096427,2.040318,0.634444,0.537158,01:14
